<a href="https://colab.research.google.com/github/dnezan/km-prediction/blob/main/KM_forecast_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **KwickMetrics Forecast Practice**
Dinesh Sreekanthan

# Data Overview
The dataset has 3 columns
<br>
*   SKU ID
*   Date
*   Number of items sold that day

By using the function `=COUNTUNIQUE(A:A)` in Google Spreadsheets, we can find the number of unique items being sold. <br>

**There are 3786 unique items being sold over a period of about 1 year (01.01.2021 - 25.11.2021)**

# Setting up Cloud Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/My Drive"

/content/drive/My Drive


In [ ]:
!mkdir km-forecast-practice
!ls

 360review	    km-forecast-practice
'Colab Notebooks'   km-forecast-practice-data.gsheet
 csvfiles	   'Untitled spreadsheet.gsheet'
 Invoice.gsheet


In [ ]:
%cd "/content/drive/My Drive/km-forecast-practice"

/content/drive/My Drive/km-forecast-practice
km-forecast-practice-data.csv


Make sure to move your dataset `km-forecast-practice.csv` to this folder.

In [ ]:
!ls

km-forecast-practice-data.csv


In [57]:
import pandas as pd
import csv
import datetime

x = datetime.datetime.now()

print(x.year)
print(x.strftime("%A"))

df=pd.read_csv('km-forecast-practice-data.csv')
df['timestamp']= pd.to_datetime(df['timestamp'])
print(df.dtypes)
df.head()


2021
Wednesday
item_id              object
timestamp    datetime64[ns]
demand                int64
dtype: object


,item_id,timestamp,demand
0,AP001AA01,2021-01-01,10
1,AP001AA01,2021-01-02,6
2,AP001AA01,2021-01-03,9
3,AP001AA01,2021-01-04,7
4,AP001AA01,2021-01-05,10


# Preprocessing the Data
The first step is to rearrange the data so that we have each column representing a day and the rows containing the SKU ID. <br> 
We must also populate the data with missing dates, which we assume have 0 items sold on that day. So the first step is to add the missing dates, then set these as 0.

In [92]:
data = [['item1', 10], ['item2', 15], ['item3', 14]]
start_date=df.timestamp[1].date()  
end_date=df.timestamp.iat[-1].date()
print('Start date is') 
print(start_date)
print('End date is')
print(end_date)

# Populating the new dataset
df_new = pd.DataFrame(data, columns = ['SKU', start_date])
x = start_date

index = 2
var=0

while var<70:
  x+= datetime.timedelta(days=1)
  df_new.insert(loc=index, column=x, value=0)
  index+=1
  var+=1
df_new.head()

Start date is
2021-01-02
End date is
2021-10-08


,SKU,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-09,2021-01-10,2021-01-11,2021-01-12,2021-01-13,2021-01-14,2021-01-15,2021-01-16,2021-01-17,2021-01-18,2021-01-19,2021-01-20,2021-01-21,2021-01-22,2021-01-23,2021-01-24,2021-01-25,2021-01-26,2021-01-27,2021-01-28,2021-01-29,2021-01-30,2021-01-31,2021-02-01,2021-02-02,2021-02-03,2021-02-04,2021-02-05,2021-02-06,2021-02-07,2021-02-08,2021-02-09,2021-02-10,2021-02-11,2021-02-12,2021-02-13,2021-02-14,2021-02-15,2021-02-16,2021-02-17,2021-02-18,2021-02-19,2021-02-20,2021-02-21,2021-02-22,2021-02-23,2021-02-24,2021-02-25,2021-02-26,2021-02-27,2021-02-28,2021-03-01,2021-03-02,2021-03-03,2021-03-04,2021-03-05,2021-03-06,2021-03-07,2021-03-08,2021-03-09,2021-03-10,2021-03-11,2021-03-12,2021-03-13
0,item1,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,item2,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,item3,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
